In [22]:
import time 
import requests, bs4
import re
import sys
import os

import pandas as pd
import numpy as np

sys.path.append("../")
from src.scraping import ScrapingSponavi

%matplotlib inline
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [13]:
ss = ScrapingSponavi()

In [14]:
url_schedule_base = "https://baseball.yahoo.co.jp/npb/schedule/"

In [31]:
date = "2020-03-21"
out_dir = os.path.join("..", "data", "html", "games")
ss.get_geme_htmls(date, out_dir)

start  2020-03-21 ==========
get html  https://baseball.yahoo.co.jp/npb/schedule/?date=2020-03-21
success
there are  2 games
get html  https://baseball.yahoo.co.jp/npb/game/2021000011/top
success
get html  https://baseball.yahoo.co.jp/npb/game/2021000012/top
success


In [ ]:
"../data/"

In [ ]:
url_schedule_day = 'https://baseball.yahoo.co.jp/npb/schedule/

In [20]:
import re

content = "https://baseball.yahoo.co.jp/npb/game/2021000011/index"
result = re.search("\d+", content)
if result:
    print(result.group())

2021000011


In [8]:
print(url)
index = re.search(r'index=\d+', url)
res = requests.get(url)

try:
    res.raise_for_status() 
except requests.exceptions.RequestException:
    print('Not Found Error in {}'.format(index.group())) #Not Foundエラーが出たらその回は終わったとみなす
    #break

soup = bs4.BeautifulSoup(res.text, "html.parser")
elems = soup.select('.bb-calendarTable__status')
elems2 = soup.select('.bb-calendarTable__venue')

#print(elems)
for elem,elem2 in zip(elems,elems2):
    url = elem.get('href')
    date = url[38:46]
    venue = elem2.text
    #print(venue)
    game_yokohama.loc[len(game_yokohama)] = [url,date,venue]
time.sleep(1)

NameError: name 'url' is not defined